# Création du Data Warehouse

## Importations

In [1]:
import pandas as pd
import sqlite3
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Cleaning

In [2]:
# Charger les données
df_agent = pd.read_csv("/content/drive/MyDrive/DATA/Hotel/agent.csv")
df_hotel = pd.read_csv("/content/drive/MyDrive/DATA/Hotel/booking.csv")

# Supprimer les réservations avec un nombre d'adultes aberrant (plus de 5)
df_hotel = df_hotel[df_hotel["adults"] <= 5]

# Supprimer les réservations avec un nombre d'enfants/bébés aberrant (plus de 5)
df_hotel = df_hotel[(df_hotel["children"] <= 5) & (df_hotel["babies"] <= 5)]

# Supprimer les Average Daily Rate (ADR) inférieur à 0
df_hotel = df_hotel[(df_hotel["adr"] >= 0)]

## Création des dimensions


In [6]:
##########
# hotel
##########
df_dim_hotel = df_hotel[['hotel']].drop_duplicates().reset_index(drop=True)
df_dim_hotel['hotel_key'] = df_dim_hotel.index + 1
df_dim_hotel.rename(columns={'hotel': 'hotel_name'}, inplace=True)

##########
# agent
##########
df_dim_agent = df_agent[['agent_id', 'starting_date', 'year', 'total_work_hours', 'total_commission']].drop_duplicates().reset_index(drop=True)
df_dim_agent['agent_key'] = df_dim_agent.index + 1

##########
# date
##########
# Extraction et nettoyage des données de date
df_dim_date = df_hotel[['arrival_date_year', 'arrival_date_month', 'arrival_date_week_number', 'arrival_date_day_of_month']].drop_duplicates()
# Conversion du mois en format numérique
df_dim_date['month'] = pd.to_datetime(df_dim_date['arrival_date_month'], format='%B').dt.month
# Création de la colonne full_date
df_dim_date['full_date'] = pd.to_datetime(df_dim_date[['arrival_date_year', 'month', 'arrival_date_day_of_month']].rename(columns={
    'arrival_date_year': 'year',
    'arrival_date_day_of_month': 'day'
}))
# Ajout des colonnes supplémentaires
df_dim_date['quarter'] = (df_dim_date['month'] - 1) // 3 + 1
df_dim_date['day_of_week'] = df_dim_date['full_date'].dt.dayofweek + 1
df_dim_date['is_weekend'] = df_dim_date['day_of_week'].isin([6, 7]).astype(int)
# Renommage final et ajout de la clé unique
df_dim_date = df_dim_date.rename(columns={
    'arrival_date_year': 'year',
    'arrival_date_week_number': 'week_number',
    'arrival_date_day_of_month': 'day'
}).reset_index(drop=True)
df_dim_date['date_key'] = df_dim_date.index + 1

##########
# customer et customer_secure
##########
# Créer df_dim_customer avec les colonnes nécessaires
df_dim_customer = df_hotel[['name', 'country', 'is_repeated_guest', 'previous_cancellations',
                            'previous_bookings_not_canceled', 'customer_type', 'required_car_parking_spaces',
                            'total_of_special_requests', 'credit_card', 'email', 'phone-number']].drop_duplicates().reset_index(drop=True)
# Ajouter une colonne 'customer_key' qui sera une clé unique pour chaque client
df_dim_customer['customer_key'] = df_dim_customer.index + 1
# Créer df_dim_customer_secure avec les informations sécurisées et la même clé 'customer_key'
df_dim_customer_secure = df_dim_customer[['customer_key', 'credit_card', 'email', 'phone-number']].drop_duplicates()

##########
# room
##########
df_dim_room = df_hotel[['reserved_room_type', 'assigned_room_type']].drop_duplicates().reset_index(drop=True)
df_dim_room['room_key'] = df_dim_room.index + 1

##########
# market
##########
df_dim_market = df_hotel[['market_segment', 'distribution_channel']].drop_duplicates().reset_index(drop=True)
df_dim_market['market_key'] = df_dim_market.index + 1

## Création de la table des fait

In [7]:
# Copie du DataFrame original
df_fact_booking = df_hotel.copy()

# Ajout de la clé étrangère pour la table `hotel`
df_fact_booking = df_fact_booking.merge(
    df_dim_hotel[['hotel_name', 'hotel_key']],  # On récupère uniquement les colonnes nécessaires
    left_on='hotel',  # Correspondance avec la colonne `hotel` de df_fact_booking
    right_on='hotel_name',
    how='left'  # Jointure à gauche pour garder toutes les réservations
)
df_fact_booking.drop(columns=['hotel_name', 'hotel'], inplace=True)

# Ajout de la clé étrangère pour la table `agent`
df_fact_booking = df_fact_booking.merge(
    df_dim_agent[['agent_id', 'year', 'agent_key']],
    left_on=['agent', 'arrival_date_year'],
    right_on=['agent_id', 'year'],
    how='left'
)
df_fact_booking.drop(columns=['agent_id', 'agent', 'year'], inplace=True)

# Conversion du mois de texte en format numérique
df_fact_booking['arrival_date_month'] = pd.to_datetime(
    df_fact_booking['arrival_date_month'], format='%B'
).dt.month

# Ajout de la clé étrangère pour la table `date`
df_fact_booking = df_fact_booking.merge(
    df_dim_date[['year', 'month', 'day', 'date_key']],
    left_on=['arrival_date_year', 'arrival_date_month', 'arrival_date_day_of_month'],
    right_on=['year', 'month', 'day'],
    how='left'
)
df_fact_booking.drop(columns=['year', 'month', 'day', 'arrival_date_year',
                              'arrival_date_month', 'arrival_date_day_of_month',
                              'arrival_date_week_number'], inplace=True)

# Ajout de la clé étrangère pour la table `room`
df_fact_booking = df_fact_booking.merge(
    df_dim_room[['reserved_room_type', 'assigned_room_type', 'room_key']],
    on=['reserved_room_type', 'assigned_room_type'],
    how='left'
)
df_fact_booking.drop(columns=['reserved_room_type', 'assigned_room_type'], inplace=True)

# Ajout de la clé étrangère pour la table `customer`
df_fact_booking = df_fact_booking.merge(
    df_dim_customer[['name', 'country', 'previous_cancellations',
                     'previous_bookings_not_canceled', 'total_of_special_requests',
                     'required_car_parking_spaces', 'is_repeated_guest', 'customer_type',
                     'credit_card', 'email', 'phone-number', 'customer_key']],
    on=['name', 'country', 'previous_cancellations',
        'previous_bookings_not_canceled', 'total_of_special_requests',
        'required_car_parking_spaces', 'is_repeated_guest', 'customer_type',
        'credit_card', 'email', 'phone-number'],
    how='left'
)
# Suppression des colonnes personnelles et redondantes après la jointure
df_fact_booking.drop(columns=['name', 'country', 'previous_cancellations',
                              'previous_bookings_not_canceled', 'total_of_special_requests',
                              'required_car_parking_spaces', 'is_repeated_guest', 'customer_type',
                              'credit_card', 'email', 'phone-number'], inplace=True)

# Ajout de la clé étrangère pour la table `market`
df_fact_booking = df_fact_booking.merge(
    df_dim_market[['market_segment', 'distribution_channel', 'market_key']],
    on=['market_segment', 'distribution_channel'],
    how='left'
)
df_fact_booking.drop(columns=['market_segment', 'distribution_channel'], inplace=True)
df_fact_booking['booking_id'] = df_fact_booking.index + 1

## Cleaning des tables créées


In [8]:
##########
# factbooking
##########
# Mise en int pour correspondre avec notre entrepôt
df_fact_booking["agent_key"] = df_fact_booking["agent_key"].fillna(-1).astype(int)
df_fact_booking["company"] = df_fact_booking["company"].fillna(-1).astype(int)
df_fact_booking['children'] = df_fact_booking['children'].astype(int)

##########
# customer
##########
# supression des colonnes inutiles
df_dim_customer.drop(columns=['credit_card','phone-number','email'], inplace=True)

##########
# agent
##########
# Ajouter une ligne pour correspondre aux agents NaN=-1
new_row = {
    'agent_key': '-1',
    'agent_id': '-1'
}
new_row_df = pd.DataFrame([new_row])
df_agent = pd.concat([df_agent, new_row_df], ignore_index=True)

## Récupération des dataframes en CSV




In [9]:
output_path = "/content/drive/MyDrive/DATA/Hotel/v2/"

# Sauvegarde des DataFrames en CSV
df_dim_agent.to_csv(output_path + "DimAgent.csv", index=False)
df_dim_date.to_csv(output_path + "DimDate.csv", index=False)
df_dim_hotel.to_csv(output_path + "DimHotel.csv", index=False)
df_dim_customer.to_csv(output_path + "DimCustomer.csv", index=False)
df_dim_customer_secure.to_csv(output_path + "DimCustomerSecure.csv", index=False)
df_dim_room.to_csv(output_path + "DimRoom.csv", index=False)
df_dim_market.to_csv(output_path + "DimMarket.csv", index=False)
df_fact_booking.to_csv(output_path + "FactBooking.csv", index=False)

## Observation

In [10]:
pd.set_option('display.max_columns', None)  # Afficher toutes les colonnes
pd.set_option('display.width', 1000)  # Ajuster la largeur d'affichage

print(df_dim_customer.head(10))
print("Nombre de lignes :",df_dim_customer.shape[0])

               name country  is_repeated_guest  previous_cancellations  previous_bookings_not_canceled customer_type  required_car_parking_spaces  total_of_special_requests  customer_key
0     Ernest Barnes     PRT                  0                       0                               0     Transient                            0                          0             1
1      Andrea Baker     PRT                  0                       0                               0     Transient                            0                          0             2
2    Rebecca Parker     GBR                  0                       0                               0     Transient                            0                          0             3
3      Laura Murray     GBR                  0                       0                               0     Transient                            0                          0             4
4       Linda Hines     GBR                  0                   